In [1]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Tongyi
import warnings
warnings.filterwarnings("ignore")

os.environ["DASHSCOPE_API_KEY"] = 'sk-69208c4b5bb04c1ab71aab0541146db9' # DAHSCOPE API KEY 表示 你的API KEY

In [2]:

llm = Tongyi() # 创建一个LLM
template = """Question: {question}
              Answer: Let's think step by step.""" # 定义一个模板

prompt = PromptTemplate.from_template(template) # 创建一个Prompt
chain = prompt | llm # 创建一个Chain
question = "你知道我们之前的对话记录吗？" # 定义一个问题

chain.invoke({"question": question})

'作为AI助手，我并不会保存或回溯我们之前的对话记录，以保护用户的隐私和数据安全。所以，我并不知道我们之前的对话内容。如果你有具体的问题或需要继续之前的话题，欢迎告诉我，我会尽力帮助你。'

In [3]:
"""我们首先直接使用模型。是 LangChain “Runnables” 的实例，这意味着它们公开了一个用于与它们交互的标准接口。要简单地调用模型，我们可以将消息列表传递给该方法。ChatModel.invoke"""
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

llm.invoke(messages)

'Ciao!'

In [5]:
"""输出解析器"""
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result=llm.invoke(messages)
parser.parse(result)
chain = llm | parser
chain.invoke(messages)

'Ciao!'

In [7]:
"""提示模板"""
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

In [8]:
"""使用LCEL将组件链接在一起"""
chain = prompt_template | llm | parser
chain.invoke({"language": "italian", "text": "hi"})

'Ciao!'

In [10]:
"""将链添加到FastAPI应用程序中"""
#!/usr/bin/env python
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Tongyi
from langserve import add_routes
from langchain_community.llms import Tongyi
os.environ["DASHSCOPE_API_KEY"] = 'sk-69208c4b5bb04c1ab71aab0541146db9'

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = Tongyi()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser

# 5. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 6. Adding chain route
add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop